## *Этап - Создание тг-бота*

#### Импорт библиотек, импорт моделей и датасетов

Для создания тг-бота использовал библиотеку telebot. Для импорта модели библиотеку pickle из предыдущего этапа. И pandas, numpy удобной работы с моделями.

In [1]:
import telebot
from telebot import types
import pickle
import pandas as pd
import numpy as np

In [2]:
# Изменил расположение токена из репозитория, чтобы скрыть его
with open('/Users/andrejgulaev/Documents/GitHub/Tokens/token.txt') as f:
    token = f.readline()
bot = telebot.TeleBot(token)

with open('model_price.pkl', 'rb') as f:
    model_price = pickle.load(f)
with open('model_links.pkl', 'rb') as f:
    model_links = pickle.load(f)

df = pd.read_csv('df_all_features.csv', index_col=0)
df_with_links = pd.read_csv('database_with_links.csv', index_col=0)

In [3]:
user_dict = {}


class Estate:
    def __init__(self, num):
        self.rooms = num
        self.square = None
        self.floor = None
        self.max_floor = None
        self.year = None
        self.balcony = None
        self.bathroom = None
        self.district = None
        
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Предоставить данные о квартире")
    markup.add(btn1)
    msg = bot.reply_to(message, """\
Здравствуйте! Я - бот, который подскажет цену на вашу квартиру в г. Пермь и отправит ссылки на похожие варинты квартир.
Мне будут необходимы следующие данные:
- количество комнат;
- площадь квартиры;
- этаж;
- этажность дома;
- год постройки дома;
- район.
""", reply_markup=markup)
    bot.register_next_step_handler(msg, first_step)


def first_step(message):
    msg = bot.reply_to(message, 'Сколько комнат в квартире?')
    bot.register_next_step_handler(msg, process_rooms_step)


def process_rooms_step(message):
    try:
        chat_id = message.chat.id
        rooms = message.text
        if not rooms.isdigit():
            msg = bot.reply_to(message, 'Количество комнат должно быть числом.\nСколько комнат в квартире?')
            bot.register_next_step_handler(msg, process_rooms_step)
            return
        estate = Estate(rooms)
        user_dict[chat_id] = estate
        msg = bot.reply_to(message, 'Какая площадь квартиры?')
        bot.register_next_step_handler(msg, process_square_step)
    except Exception as e:
        bot.reply_to(message, 'Ошибка process_rooms_step')


def process_square_step(message):
    try:
        chat_id = message.chat.id
        square = message.text
        if not square.isdigit():
            msg = bot.reply_to(message, 'Площадь должна быть числом. Какая площадь квартиры?')
            bot.register_next_step_handler(msg, process_square_step)
            return
        estate = user_dict[chat_id]
        estate.square = square
        msg = bot.reply_to(message, 'Какой этаж?')
        bot.register_next_step_handler(msg, process_max_floor_step)
    except Exception as e:
        bot.reply_to(message, 'Ошибка process_square_step')


def process_max_floor_step(message):
    try:
        chat_id = message.chat.id
        floor = message.text
        if not floor.isdigit():
            msg = bot.reply_to(message, 'Этаж должен быть числом. Какой этаж?')
            bot.register_next_step_handler(msg, process_max_floor_step)
            return
        estate = user_dict[chat_id]
        estate.floor = floor
        msg = bot.reply_to(message, 'Сколько этажей дом?')
        bot.register_next_step_handler(msg, process_year_step)
    except Exception as e:
        bot.reply_to(message, 'Ошибка process_max_floor_step')
        
        
def process_year_step(message):
    try:
        chat_id = message.chat.id
        max_floor = message.text
        if not max_floor.isdigit():
            msg = bot.reply_to(message, 'Этажность должна быть числом. Сколько этажей дом?')
            bot.register_next_step_handler(msg, process_balcony_step)
            return
        estate = user_dict[chat_id]
        estate.max_floor = max_floor
        msg = bot.reply_to(message, 'Какого года дом?')
        bot.register_next_step_handler(msg, process_balcony_step)
    except Exception as e:
        bot.reply_to(message, 'Ошибка process_year_step')

        
def process_balcony_step(message):
    try:
        chat_id = message.chat.id
        year = message.text
        if not year.isdigit():
            msg = bot.reply_to(message, 'Год должен быть числом. Какой год?')
            bot.register_next_step_handler(msg, process_ending_step)
            return
        estate = user_dict[chat_id]
        estate.year = year
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('балкон', 'лоджия')
        msg = bot.reply_to(message, 'Балкон или лоджия?', reply_markup=markup)
        bot.register_next_step_handler(msg, process_bathroom_step)
    except Exception as e:
        bot.reply_to(message, 'Ошибка process_balcony_step')

def process_bathroom_step(message):
    try:
        chat_id = message.chat.id
        balcony = message.text
        estate = user_dict[chat_id]
        if (balcony == u'балкон') or (balcony == u'лоджия'):
            estate.balcony = balcony
        else:
            raise Exception()
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('раздельный', 'совмещенный')
        msg = bot.reply_to(message, 'Санузел раздельный или совмещенный?', reply_markup=markup)
        bot.register_next_step_handler(msg, process_district_step)
    except Exception as e:
        bot.reply_to(message, 'Ошибка process_bathroom_step')

        
def process_district_step(message):
    try:
        chat_id = message.chat.id
        bathroom = message.text
        estate = user_dict[chat_id]
        if (bathroom == u'раздельный') or (bathroom == u'совмещенный'):
            estate.bathroom = bathroom
        else:
            raise Exception()
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Свердловский', 'Мотовилихинский', 'Индустриальный', 'Дзержинский', 'Орджоникидзевский', 'Кировский', 'Ленинский')
        msg = bot.reply_to(message, 'Какой район?', reply_markup=markup)
        bot.register_next_step_handler(msg, process_ending_step)
    except Exception as e:
        bot.reply_to(message, 'Ошибка process_district_step')   
        
        
def process_ending_step(message):
    try:
        chat_id = message.chat.id
        district = message.text
        estate = user_dict[chat_id]
        estate.district = district
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Узнать цену')
        msg = bot.reply_to(message, f'''Ваши данные по квартире:
- количество комнат: {estate.rooms}
- площадь квартиры: {estate.square}
- этаж: {estate.floor}
- этажность дома: {estate.max_floor}
- год постройки дома: {estate.year}
- {estate.balcony}
- санузел: {estate.bathroom}
- район: {estate.district}''', reply_markup=markup)
        bot.register_next_step_handler(msg, predict_price)
    except Exception as e:
        bot.reply_to(message, 'Ошибка process_ending_step')


def predict_price(message):
    try:
        chat_id = message.chat.id
        estate = user_dict[chat_id]
        
        X_pred = pd.DataFrame({'Rooms': [int(estate.rooms)], 
                               'Area': [float(estate.square)], 
                               'Floor': [int(estate.floor)], 
                               'Max_house_floor': [int(estate.max_floor)],
                               'Year_built': [int(estate.year)],
                               'balcony_or_loggia_балкон': [0],
                               'balcony_or_loggia_лоджия': [0],
                               'Bathroom_раздельный': [0],
                               'Bathroom_совмещенный': [0],
                               'District_Дзержинский': [0],
                               'District_Индустриальный': [0],
                               'District_Кировский': [0],
                               'District_Ленинский': [0],
                               'District_Мотовилихинский': [0],
                               'District_Орджоникидзевский': [0],
                               'District_Свердловский': [0]})
        
        if estate.balcony == 'балкон':
            X_pred['balcony_or_loggia_балкон'] = 1
        else:
            X_pred['balcony_or_loggia_лоджия'] = 1
        
        if estate.bathroom == 'раздельный':
            X_pred['Bathroom_раздельный'] = 1
        else:
            X_pred['Bathroom_совмещенный'] = 1
        
        if estate.district == 'Дзержинский':
            X_pred['District_Дзержинский'] = 1
        elif estate.district == 'Индустриальный':
            X_pred['District_Индустриальный'] = 1
        elif estate.district == 'Кировский':
            X_pred['District_Кировский'] = 1
        elif estate.district == 'Ленинский':
            X_pred['District_Ленинский'] = 1
        elif estate.district == 'Мотовилихинский':
            X_pred['District_Мотовилихинский'] = 1
        elif estate.district == 'Орджоникидзевский':
            X_pred['District_Орджоникидзевский'] = 1
        elif estate.district == 'Свердловский':
            X_pred['District_Свердловский'] = 1
        
        neighbors = model_links.best_estimator_[1].kneighbors(X_pred, return_distance=False)
        links = []
        for i in neighbors[0]:
            if df_with_links.iloc[i, 1] == X_pred.iloc[0, 0]:
                links.append(df_with_links.iloc[i, -1])
                
                
        for column in X_pred.columns[:5]:
            X_pred[column] = np.log(X_pred[column])
        price = int(np.exp(model_price.predict(X_pred)[0]))
        price_if = int(price)
        price -= price % -10000
        price = str(price)
        if len(price) == 7:
            price = price[0] + ' ' + price[1:4] + ' ' + price[4:]
        elif len(price) == 8:
            price = price[:2] + ' ' + price[1:4] + ' ' + price[4:]
        
        bot.send_message(chat_id, f'''Ориентировочная стоимость вашей квартиры: <b>{price}</b> руб.
Похожие объявления:
{' '.join(links)}''', parse_mode='html')
    except Exception as e:
        bot.reply_to(message, 'Ошибка predict_price')

In [4]:
bot.polling(none_stop=True)